# **Malware Detector**

Copy data in from drive:

In [ ]:
!rm -dr data
!cp /content/drive/MyDrive/ml_sec_hw_data.zip /content/ml_sec_hw_data.zip
!unzip -q -P M4bQPt4a2us9L2 /content/ml_sec_hw_data.zip

rm: cannot remove 'data': No such file or directory


This is put in automatically by allowing drive, but the folder may have been mounted already.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Imports and weights & biases logging

In [ ]:
use_wandb = False
if use_wandb:
  !pip install wandb -qU
  import wandb
  wandb.login()


import os
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss # This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.
import numpy as np
from time import sleep
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam



GPU support

In [ ]:
# https://stackoverflow.com/a/63302819/11080152
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

---

## This function makes the padding and averaging of input bytes.

In [ ]:
def process_binary(content, s):
    l = len(content)
    if l==s: return content
    if l < s:
        padded_content = np.pad(content, (0, s - l)) # add zeros to reach 1*s length
    else:
        group_size = np.ceil(l / s).astype(int) # this many bytes will be averaged together into one
        padded_length = group_size * s # new length = rounding up the original lenght to a whole multiple of s
        padded_content = np.pad(content, (0, padded_length - l)) # add zeros to reach ceil(l/s) lenght
        reshaped = padded_content.reshape(-1, group_size) # [X,X,X,...] -> [[<group_size>],[<group_size>],...]
        averaged_content = reshaped.mean(axis=1) # [[A,B,C],[D,E,F],...] -> [avg1,avg2,...]
        padded_content = averaged_content
    normalized = padded_content / 255 # so every number will be between 0 and 1.
    return normalized

## Loading in the files with the help of Torch classes

In [ ]:
class BinaryFilesDataset(Dataset):
    def __init__(self, directory, s):
        self.files = []
        self.labels = []
        self.s = s

        for label, kind in enumerate(['benign', 'malware']): # this defines that benign is 0, and malware is 1.
            path = os.path.join(directory, kind)
            files = [os.path.join(path, file) for file in os.listdir(path)] # get the list of files in the current path
            self.files.extend(files) # add the files to the main list
            self.labels.extend([label] * len(files)) # in one folder, all binaries are getting the same label, add the label as many times as many files are added

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        with open(file_path, 'rb') as f:
          content = np.fromfile(f, dtype=np.uint8)
        return torch.tensor(process_binary(content, self.s), dtype=torch.float), self.labels[idx]

For the attack later, we will use this class to get more data out from the dataset:

In [ ]:
class BinaryFilesDatasetWithSuffixAndMask(Dataset):
    def __init__(self, directory, s, suffix_percentage):
        self.files = []
        self.labels = []
        self.s = s
        self.suffix_percentage = suffix_percentage

        for label, kind in enumerate(['benign', 'malware']): # this defines that benign is 0, and malware is 1.
            path = os.path.join(directory, kind)
            files = [os.path.join(path, file) for file in os.listdir(path)] # get the list of files in the current path
            self.files.extend(files) # add the files to the main list
            self.labels.extend([label] * len(files)) # in one folder, all binaries are getting the same label, add the label as many times as many files are added

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        label = self.labels[idx]

        with open(file_path, 'rb') as f:
            content = np.fromfile(f, dtype=np.uint8)

        original_length = len(content)
        adversarial_suffix_len = int(np.ceil(original_length * self.suffix_percentage))

        # Process without suffix
        data_without_suffix = process_binary(content, self.s)

        # Add suffix
        suffixed_content = np.pad(content, (0, adversarial_suffix_len))
        data_with_suffix = process_binary(suffixed_content, self.s)

        # Calculate mask
        total_length_with_padding = np.ceil((original_length + adversarial_suffix_len) / self.s) * self.s
        padded_length = total_length_with_padding - (original_length + adversarial_suffix_len)
        fully_modifiable_bytes = total_length_with_padding - padded_length # Exclude padding-influenced bytes

        group_size = np.ceil(total_length_with_padding / self.s).astype(int)
        num_groups = len(data_with_suffix)
        modifiable_mask = np.zeros(num_groups, dtype=np.uint8)

        modifiable_start_group = np.ceil((original_length / group_size)).astype(int) # Last group might be influenced by padding -> ceil it.
        modifiable_end_group = np.floor(fully_modifiable_bytes / group_size).astype(int) # floor is for exclusivity on the end

        modifiable_mask[modifiable_start_group:modifiable_end_group] = 1 # modifiable bytes get the 1 number, others kept at 0

        return torch.tensor(data_without_suffix, dtype=torch.float), torch.tensor(data_with_suffix, dtype=torch.float), torch.tensor(modifiable_mask, dtype=torch.float), torch.tensor(label, dtype=torch.long), os.path.basename(file_path.split("/")[-1]), original_length, group_size

## The model

In [ ]:
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

class ExecutableClassifier(nn.Module):
    def __init__(self, s):
        super(ExecutableClassifier, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=(10,), stride=(1,)),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False) # instructions were unclear, decision was made to have stride=4 instead of dobuling linear in feature size.
        )

        self.linear = nn.Linear(in_features=65488, out_features=1, bias=True)
        # self.out = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        # x = self.out(x) # BCEWithLogitsLoss will apply sigmoid later.
        return x

## Train the model

In [ ]:
def train_model(model, train_loader, num_epochs, lr):
    optimizer = Adam(model.parameters(), lr)
    criterion = BCEWithLogitsLoss()

    if use_wandb:
      wandb.watch(model, log='all')

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).unsqueeze(1), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader) # to correct batched runs divide by batch size

        # Log the average loss and current epoch number to wandb
        if use_wandb:
          wandb.log({"epoch": epoch, "loss": avg_loss})

        print(f"Epoch ", epoch+1, "Loss: ",avg_loss)
    if use_wandb:
      wandb.unwatch()

if use_wandb:
  wandb.init(project='hf1')

train_dataset = BinaryFilesDataset('data/train', s=2**14) # s is 2^14
train_loader = DataLoader(train_dataset, batch_size=220, shuffle=True)

model = ExecutableClassifier(s=2**14).to(device)
print(model)

train_model(model, train_loader, num_epochs=6, lr=0.001)

if use_wandb:
  wandb.finish()

ExecutableClassifier(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(10,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (linear): Linear(in_features=65488, out_features=1, bias=True)
)
Epoch  1 Loss:  0.947080135345459
Epoch  2 Loss:  0.3860979424789548
Epoch  3 Loss:  0.20726231671869755
Epoch  4 Loss:  0.15704669244587421
Epoch  5 Loss:  0.13449661200866103
Epoch  6 Loss:  0.11198852583765984


## Test the model

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

def test_model_with_metrics(model, test_loader):
    model.eval()
    predictions = []
    actuals = []
    predicted_probs = []

    with torch.no_grad():
      for inputs, labels in test_loader:
          inputs, labels = inputs.to(device).unsqueeze(1), labels.to(device).float().unsqueeze(1)
          outputs = model(inputs)
          prob = torch.sigmoid(outputs) # Probability of being being malware
          predicted_classes = (prob > 0.5).long()  # Applying threshold to convert to binary class

          # move back to cpu in case device was GPU
          predicted_probs.extend(prob.cpu().numpy().flatten())
          predictions.extend(predicted_classes.cpu().numpy())
          actuals.extend(labels.cpu().numpy())

    accuracy = accuracy_score(actuals, predictions)
    auc = roc_auc_score(actuals, predicted_probs)  # AUC calculation

    # Confusion matrix elements: tn, fp, fn, tp
    tn, fp, fn, tp = confusion_matrix(actuals, predictions).ravel()

    # Calculating the metrics
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    fpr = fp / (tn + fp)  # False Positive Rate
    fnr = fn / (tp + fn)  # False Negative Rate

    return accuracy, auc, tpr, tnr, fpr, fnr

test_dataset = BinaryFilesDataset('data/test', s=2**14)
test_loader = DataLoader(test_dataset)

# Test the model and print the metrics
accuracy, auc, tpr, tnr, fpr, fnr = test_model_with_metrics(model, test_loader)
print(f"Test Accuracy: ", accuracy*100, "%")
print(f"AUC: ",auc)
print(f"TPR: ",tpr)
print(f"TNR: ",tnr)
print(f"FPR: ",fpr)
print(f"FNR: ",fnr)

Test Accuracy:  96.41025641025641 %
AUC:  0.9949769888231426
TPR:  0.9692307692307692
TNR:  0.958974358974359
FPR:  0.041025641025641026
FNR:  0.03076923076923077


Accuracy is good. if epoch number increased/batch size decreased, it could have even better test results than shown here.

---

# **Attack**

In [ ]:
import torch.optim as optim

# Define the PGD attack function avoiding the dimension expansion error
def pgd_attack(model, original_input, original_label, suffix_byte_mask, num_max_iterations, lr):
    target_probability = 1.0 - original_label  # Aim for the opposite class
    delta = torch.zeros_like(original_input, requires_grad=True)

    opt = optim.SGD([delta], lr=lr)

    current_loss = 1
    current_iterations = 0
    while(current_loss > 0.5 and current_iterations < num_max_iterations): # iterate until it is good enough or itetation limit reached
      current_iterations = current_iterations + 1

      modified_input = (original_input + delta * suffix_byte_mask).to(device).unsqueeze(0).unsqueeze(1) # only change the changeable bytes with delta

      pred = model(modified_input)
      loss = nn.BCEWithLogitsLoss()(pred, target_probability.to(device).unsqueeze(0))
      current_loss = loss.item()

      # if t % int((num_iterations/10)) == 0:
      #     print(f"Iteration {t}, Loss: {loss.item()}")

      opt.zero_grad()
      loss.backward()

      with torch.no_grad():
          delta.grad *= suffix_byte_mask # zero out the not changeable bytes
          opt.step()

    with torch.no_grad():
        delta *= suffix_byte_mask # zero out the not changeable bytes

    final_input = original_input + delta
    final_input_shaped = final_input.unsqueeze(0).unsqueeze(1)

    return final_input_shaped.detach(), delta.detach(), current_iterations

Try it out:

In [ ]:
model.eval()

lr_=0.25
num_max_iterations=3000
for suffix_pcnt in [0.05,0.1,0.15,0.2]:
  dataset = BinaryFilesDatasetWithSuffixAndMask(directory='data/victim', s=2**14, suffix_percentage=suffix_pcnt)
  loader = DataLoader(dataset,shuffle=False)

  PGD_success = 0
  PGD_fail = 0
  avg_suffix_len = 0

  # Search for a malware sample
  for data_without_suffix, data_with_suffix, modifiable_mask, label, file_path, original_length, group_size in loader:
      if label.item() == 1:  # looking for malwares
          original_input = data_with_suffix
          suffix_byte_mask = modifiable_mask  # modifiable_mask indicating modifiable bytes
          original_label = label

          # Make sure to convert the label to a float for consistency
          original_label = original_label.float()

          # Run PGD attack
          perturbed_input, delta, iterations = pgd_attack(
              model=model,
              original_input=original_input.squeeze(0),
              original_label=original_label,
              suffix_byte_mask=suffix_byte_mask.squeeze(0),
              num_max_iterations=num_max_iterations,
              lr=lr_
          )

          # Get predictions for original and perturbed inputs to compare
          original_pred = torch.sigmoid(model(original_input.to(device).unsqueeze(0))).item()
          perturbed_pred = torch.sigmoid(model(perturbed_input.to(device))).item()

          misclassification_happened = perturbed_pred < 0.5

          print(iterations, "; ", suffix_pcnt, "; ", file_path[0], "; ", modifiable_mask.sum().int().item(), "; ", (modifiable_mask.sum() * group_size / original_length).item(), "; ", original_pred, "; ", perturbed_pred, "; ", misclassification_happened)

          avg_suffix_len += modifiable_mask.sum() * group_size / original_length

          if (misclassification_happened):
            PGD_success = PGD_success + 1
          else:
            PGD_fail = PGD_fail + 1

  print("learning rate:",lr_, "; suffix_length:", suffix_pcnt, "bytes (avg ",(avg_suffix_len/(PGD_success+PGD_fail)).item(), " bytes); PGD_success:", PGD_success, "; PGD_fail:", PGD_fail)

87 ;  0.05 ;  54d2b0edff1728d14da5c59d95cdd36553df082733d75126dd6bcbb8ed102fdb ;  758 ;  0.049935951828956604 ;  0.8632876873016357 ;  0.38397860527038574 ;  True
2291 ;  0.05 ;  31c387b20fa92966292b0771d1eb265599d7c2835b87808c6d46a84c7d38a7b8 ;  691 ;  0.049908511340618134 ;  0.9460225701332092 ;  0.39314591884613037 ;  True
545 ;  0.05 ;  790d7efcdc2300f0046a2a2c69925cd6e3d9bf5e1452e7e8560669fcae5690e1 ;  727 ;  0.049979377537965775 ;  0.987644374370575 ;  0.3920239210128784 ;  True
81 ;  0.05 ;  10af6612ead6097133d197e6021ed8699bc660076360039033f4942fab5da340 ;  771 ;  0.04995817691087723 ;  0.9311416149139404 ;  0.38186976313591003 ;  True
669 ;  0.05 ;  f537b4d2fd535df7ab069390813c6ff09122479a35f135e3f036ae5de40ad007 ;  707 ;  0.04994449391961098 ;  0.7691749930381775 ;  0.39276614785194397 ;  True
2464 ;  0.05 ;  7b66f2eb9e257d77fc0fca2e463fba88e60902fecc126fc04d3e2e32e630606c ;  693 ;  0.049981970340013504 ;  0.9748874306678772 ;  0.39313098788261414 ;  True
196 ;  0.05 ;  b44e4


```
learning rate: 0.25 ; suffix_length: 0.05 bytes (avg  0.04995940998196602  bytes); PGD_success: 39 ; PGD_fail: 11
learning rate: 0.25 ; suffix_length: 0.1 bytes (avg  0.09995482116937637  bytes); PGD_success: 43 ; PGD_fail: 7
learning rate: 0.25 ; suffix_length: 0.15 bytes (avg  0.14994782209396362  bytes); PGD_success: 43 ; PGD_fail: 7
learning rate: 0.25 ; suffix_length: 0.2 bytes (avg  0.19995646178722382  bytes); PGD_success: 48 ; PGD_fail: 2
```

- 5% -> **78%**
- 10%  -> **86%**
- 15% -> **86%**
- 20%  -> **96%**

As suffx length increases, the success rate increases as well. (in this very run this is not exactly the case, but the trend was clear during testing. we also tried different learning rates, 0.25 was near optimal (0.2 and 0.3 was worse usually))

## Random test

In [ ]:
def random_fill_masked_bytes(original_input, suffix_byte_mask):
    # Initialize delta as zeros with the same shape as the original input
    delta = torch.zeros_like(original_input)
    # https://pytorch.org/docs/stable/generated/torch.rand_like.html
    random_values = torch.rand_like(original_input) # random values between 0 and 1 for delta using uniform distribution
    delta = random_values * suffix_byte_mask # Apply the mask - only keep random values for maskable bytes
    final_input = original_input + delta # updated bytes are all zeros, so no need to clear the area first, just add random delta to it.
    return final_input, delta

In [ ]:
for suffix_pcnt in [0.05,0.1,0.15,0.2,0.3,0.4,0.6,0.8]:
  dataset = BinaryFilesDatasetWithSuffixAndMask(directory='data/victim', s=2**14, suffix_percentage=suffix_pcnt)
  loader = DataLoader(dataset,shuffle=False)

  PGD_success = 0
  PGD_fail = 0
  avg_suffix_len = 0
  avg_accuracy = 0

  # Search for a malware sample
  for data_without_suffix, data_with_suffix, modifiable_mask, label, file_path, original_length, group_size in loader:
      if label.item() == 1: # looking for malwares
          original_input = data_with_suffix
          suffix_byte_mask = modifiable_mask  # modifiable_mask indicating modifiable bytes
          original_label = label

          original_label = original_label.float()

          # Run random attack
          perturbed_input, delta = random_fill_masked_bytes( original_input=original_input.squeeze(0), suffix_byte_mask=suffix_byte_mask.squeeze(0) )

          # Get predictions for original and perturbed inputs to compare
          original_pred = torch.sigmoid(model(original_input.to(device).unsqueeze(0))).item()
          perturbed_pred = torch.sigmoid(model(perturbed_input.to(device).unsqueeze(0).unsqueeze(0)))

          misclassification_happened = ( perturbed_pred < 0.5 ).item()

          print(iterations, "; ", suffix_pcnt, "; ", file_path[0], "; ", modifiable_mask.sum().int().item(), "; ", (modifiable_mask.sum() * group_size / original_length).item(), "; ", original_pred, "; ", perturbed_pred.item(), "; ", misclassification_happened)

          avg_suffix_len += modifiable_mask.sum() * group_size / original_length

          if(original_pred>0.5):
            avg_accuracy = avg_accuracy + 1
          if (misclassification_happened):
            PGD_success = PGD_success + 1
          else:
            PGD_fail = PGD_fail + 1

  print("learning rate:",lr_, "; suffix_length:", suffix_pcnt, "bytes (avg ",(avg_suffix_len/(PGD_success+PGD_fail)).item(), " bytes); PGD_success:", PGD_success, "; PGD_fail:", PGD_fail, "; Avg accuracy for zeros: ", avg_accuracy/(PGD_success+PGD_fail))

228 ;  0.05 ;  54d2b0edff1728d14da5c59d95cdd36553df082733d75126dd6bcbb8ed102fdb ;  758 ;  0.049935951828956604 ;  0.8632876873016357 ;  0.5330973267555237 ;  False
228 ;  0.05 ;  31c387b20fa92966292b0771d1eb265599d7c2835b87808c6d46a84c7d38a7b8 ;  691 ;  0.049908511340618134 ;  0.9460225701332092 ;  0.9663752913475037 ;  False
228 ;  0.05 ;  790d7efcdc2300f0046a2a2c69925cd6e3d9bf5e1452e7e8560669fcae5690e1 ;  727 ;  0.049979377537965775 ;  0.987644374370575 ;  0.9909330606460571 ;  False
228 ;  0.05 ;  10af6612ead6097133d197e6021ed8699bc660076360039033f4942fab5da340 ;  771 ;  0.04995817691087723 ;  0.9311416149139404 ;  0.5697913765907288 ;  False
228 ;  0.05 ;  f537b4d2fd535df7ab069390813c6ff09122479a35f135e3f036ae5de40ad007 ;  707 ;  0.04994449391961098 ;  0.7691749930381775 ;  0.8270352482795715 ;  False
228 ;  0.05 ;  7b66f2eb9e257d77fc0fca2e463fba88e60902fecc126fc04d3e2e32e630606c ;  693 ;  0.049981970340013504 ;  0.9748874306678772 ;  0.9824371337890625 ;  False
228 ;  0.05 ;  b44e

```
learning rate: 0.25 ; suffix_length: 0.05 bytes (avg  0.04995940998196602  bytes); PGD_success: 4 ; PGD_fail: 46
learning rate: 0.25 ; suffix_length: 0.1 bytes (avg  0.09995482116937637  bytes); PGD_success: 6 ; PGD_fail: 44
learning rate: 0.25 ; suffix_length: 0.15 bytes (avg  0.14994782209396362  bytes); PGD_success: 11 ; PGD_fail: 39
learning rate: 0.25 ; suffix_length: 0.2 bytes (avg  0.19995646178722382  bytes); PGD_success: 7 ; PGD_fail: 43
```

- 5% -> **8%**
- 10%  -> **12%**
- 15% -> **22%**
- 20%  -> **14%**


Random noise did not yield good results, and it even seemed in previous runs that as the length of added noise increases, the success rate lowers.


---

For test/fun try to misclassify benign victims as malware

In [ ]:
model.eval()

lr_=0.25
num_max_iterations=3000
for suffix_pcnt in [0.05,0.1,0.15,0.2]:
  dataset = BinaryFilesDatasetWithSuffixAndMask(directory='data/victim', s=2**14, suffix_percentage=suffix_pcnt)
  loader = DataLoader(dataset,shuffle=False)

  PGD_success = 0
  PGD_fail = 0
  avg_suffix_len = 0

  # Search for a benigins sample
  for data_without_suffix, data_with_suffix, modifiable_mask, label, file_path, original_length, group_size in loader:
      if label.item() == 0:  # ! looking for benigins now
          original_input = data_with_suffix
          suffix_byte_mask = modifiable_mask
          original_label = label

          original_label = original_label.float()

          # Run PGD attack
          perturbed_input, delta, iterations = pgd_attack(
              model=model,
              original_input=original_input.squeeze(0),
              original_label=original_label,
              suffix_byte_mask=suffix_byte_mask.squeeze(0),
              num_max_iterations=num_max_iterations,
              lr=lr_
          )

          # Get predictions for original and perturbed inputs to compare
          original_pred = torch.sigmoid(model(original_input.to(device).unsqueeze(0))).item()
          perturbed_pred = torch.sigmoid(model(perturbed_input.to(device))).item()

          misclassification_happened = perturbed_pred > 0.5

          print(iterations, "; ", suffix_pcnt, "; ", file_path[0], "; ", modifiable_mask.sum().int().item(), "; ", (modifiable_mask.sum() * group_size / original_length).item(), "; ", original_pred, "; ", perturbed_pred, "; ", misclassification_happened)

          avg_suffix_len += modifiable_mask.sum() * group_size / original_length

          if (misclassification_happened):
            PGD_success = PGD_success + 1
          else:
            PGD_fail = PGD_fail + 1

  print("learning rate:",lr_, "; suffix_length:", suffix_pcnt, "bytes (avg ",(avg_suffix_len/(PGD_success+PGD_fail)).item(), " bytes); PGD_success:", PGD_success, "; PGD_fail:", PGD_fail)

3000 ;  0.05 ;  75bc647100e6ee5c24b0ca3baafed38f439e63a08170bb480ca953f24e2737b4 ;  779 ;  0.049971774220466614 ;  0.27788326144218445 ;  0.5964540839195251 ;  True
199 ;  0.05 ;  daae12e654e916f5bec506e2beefb42db8219197a9b3576835cd2bc6db0d06d3 ;  664 ;  0.04997929930686951 ;  0.0023241264279931784 ;  0.6173883080482483 ;  True
3000 ;  0.05 ;  655ea57c28f1b938359146a885e54f0992b02856461383502357b1a0e4025bcc ;  760 ;  0.0499478355050087 ;  0.09665806591510773 ;  0.5229609608650208 ;  True
3000 ;  0.05 ;  c091d9dbc7517666f39e94a6903afaaf0ee6bff7cc227a2ff29a1e222ecba749 ;  760 ;  0.049927081912755966 ;  0.02390477992594242 ;  0.4118695855140686 ;  False
2273 ;  0.05 ;  156d2c345b40543b86b8de0f0610f4cfe4bf9ceed4fe7729b0290b2882c33f9b ;  494 ;  0.05002531781792641 ;  0.0009243635577149689 ;  0.6071246862411499 ;  True
1291 ;  0.05 ;  1289b3847594466942d82d78e818ccb5ab4fd78f35fec9b0b4e987372f1f3b1f ;  735 ;  0.04989421367645264 ;  0.03485419228672981 ;  0.6078366041183472 ;  True
3000 ;  0.0

```
learning rate: 0.25 ; suffix_length: 0.05 bytes (avg  0.049979038536548615  bytes); PGD_success: 41 ; PGD_fail: 9
learning rate: 0.25 ; suffix_length: 0.1 bytes (avg  0.09997206926345825  bytes); PGD_success: 48 ; PGD_fail: 2
learning rate: 0.25 ; suffix_length: 0.15 bytes (avg  0.1499818116426468  bytes); PGD_success: 48 ; PGD_fail: 2
learning rate: 0.25 ; suffix_length: 0.2 bytes (avg  0.1999654620885849  bytes); PGD_success: 48 ; PGD_fail: 2
```

- 5% -> **82%**
- 10%  -> **96%**
- 15% -> **96%**
- 20%  -> **96%**

This way yielded better results than the original task on all suffix percentage rates. The expected result was to get the same numbers.

It could be that the boundary is "closer" to benign, making them easier to misclassify.